In [ ]:
asdf

# Imports

In [1]:
import pathlib as pl
import nesdis_aws
import nesdis_gml_synergy.satlab as ngs
import atmPy.data_archives.NOAA_ESRL_GMD_GRAD.surfrad.surfrad as surfrad
import os
os.getpid()

7468

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# warnings.filterwarnings("default")

In [4]:
plt.rcParams['figure.dpi'] = 200

# download some data

In [ ]:
reload(nesdis_aws.nesdis_aws)
reload(nesdis_aws)

In [ ]:
# product = 'ACHa' #FDC
path2folder_local=f'/mnt/telg/data/nesdis/goes/ABI_L2_{product}/'
pl.Path(path2folder_local).mkdir()
query = nesdis_aws.AwsQuery(path2folder_local=path2folder_local,
                            satellite='16',
                            product=f'ABI-L2-{product}',
                            scan_sector='C',
                            # start='2020-09-30 18:00:00',
                            # end='2020-09-30 19:35:00',
                            start='2018-08-01 18:00:00',
                            # end='2018-11-01 00:00:00',
                            end='2018-08-01 18:05:00',
                            verbose = True
                           )


In [ ]:
query.workplan

In [ ]:
print(query.info_on_current_query())

In [ ]:
query.product_available_since

In [ ]:
query.download()


# open

In [ ]:
import atmPy.data_archives.NOAA_ESRL_GMD_GRAD.surfrad.surfrad as surfrad
stations = surfrad.network.stations

In [ ]:
reload(ngs)

In [ ]:
# sat_ds = ngs.open_file('/mnt/telg/data/nesdis/goes/ABI_L2_ADP/OR_ABI-L2-ADPC-M6_G16_s20211431801169_e20211431803542_c20211431804531.nc', verbose=True)

In [ ]:
sat_ds.valid_2D_variables

In [ ]:
list(sat_ds.ds.variables)

In [ ]:
proj = sat_ds.project_on_sites(stations)

In [ ]:
proj.projection2area.sel(site = 'BND', stats = 'mean', radius = 50, 
                         # data_quality = 'good'
                        )

In [ ]:
proj.projection2point

# Plot on map

In [ ]:
scale = 0.5
width=2800000 *scale
height=1700000 * scale
a, bmap = surfrad.network.stations.Table_Mountain.plot(width=width, height=height)
bmap, pc, cb = sat_ds.plot('Temp', bmap = bmap)
bmap.drawcounties()
# f = plt.gcf()
# pc.set_clim(vmax= 10)

# Download and project

In [5]:
import pathlib as pl
import nesdis_aws
import nesdis_gml_synergy.satlab as ngs
import atmPy.data_archives.NOAA_ESRL_GMD_GRAD.surfrad.surfrad as surfrad
import os
os.getpid()

7468

In [6]:
import atmPy.data_archives.NOAA_ESRL_GMD_GRAD.surfrad.surfrad as surfrad

stations = surfrad.network.stations

In [ ]:
# def function(row, stations):#, alt_var = None):
#     # if isinstance(alt_var, type(None)):
#     #     alt_var = variable
#     # read the file
#     ngsinst = ngs.open_file(row.path2file_local)
    
#     # project to stations
#     projection = ngsinst.project_on_sites(stations)

#     # merge closest gridpoint and area
#     point = projection.projection2point.copy()#.sel(site = 'TBL')
#     point['DQF'] = point.DQF.astype(int) # for some reason this was float64... because there are some nans in there
    
#     # change var names to distinguish from area
#     for var in ngsinst.valid_2D_variables:
#         point = point.rename({var: f'{var}_on_pixel',})
#     point = point.rename({'DQF': 'DQF_on_pixel'})
    
#     # merge aerea and point
#     ds = projection.projection2area.merge(point)#.rename({alt_var: f'{alt_var}_on_pixel', 'DQF': 'DQF_on_pixel'}))
    
#     # add a time stamp
#     dt = pd.Series([pd.to_datetime(ngsinst.ds.attrs['time_coverage_start']), pd.to_datetime(ngsinst.ds.attrs['time_coverage_end'])]).mean().to_datetime64()
#     ds = ds.expand_dims({'datetime': [dt]}, )
    
#     # there was another time coordinate without dimention though ... dropit
#     ds = ds.drop_vars('t')

#     # global attribute
#     ds.attrs['info'] = ('This file contains a projection of satellite data onto SURFRAD sites.\n'
#                          'It includes the closest pixel data as well as the average over circular\n'
#                          'areas with various radii. Note, for the averaged data only data is\n'
#                          'considered with a qulity flag given by the prooduct class in the\n'
#                          'nesdis_gml_synergy package.')
    

#     # save2file
#     ds.to_netcdf(row.path2file_local_processed)
#     # Memory kept on piling up -> maybe a cleanup will help
#     ds.close()
#     ds = None
#     ngsinst.ds.close()
#     ngsinst = None
#     return 

In [13]:
product = 'ACM'
# alt_var = 'BCM'# None, if the varialbe name is not equal to product name
path2processed =  f'/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_{product}_projected2surfrad/'
query = nesdis_aws.nesdis_aws.AwsQuery(
                                        path2folder_local=f'/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_{product}/',
                                        satellite='16',
                                        product=f'ABI-L2-{product}',
                                        scan_sector='C',
                                        # start='2020-03-01 18:00:00',
                                        # end='2020-03-01 18:05:00',
                                        #
                                        # CPEX spring 2020
                                        # start='2020-03-01 00:00:00',
                                        # end='2020-06-01 00:00:00',
                                        #
                                        # CPEX original
                                        # start='2018-08-01 00:00:00',
                                        # end='2018-11-01 00:00:00',
                                        #
                                        # all time
                                        start='2017-04-20 13:00:00',
                                        end='2017-04-20 13:05:00',
                                        # end='2022-08-01 00:00:00',
                                        process = dict(#concatenate = 'daily',
                                                       function = lambda row: ngs.projection_function(row, stations), #,, alt_var),
                                                       prefix = f'ABI_L2_{product}_projected2surfrad',
                                                       path2processed = path2processed,
                                                        )
                                        # keep_files=True
                                      )

# pl.Path(f'/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_{product}').mkdir()
# pl.Path(f'/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_{product}_projected2surfrad/').mkdir()

In [14]:
query.workplan

,path2file_aws,path2file_local,path2file_local_processed


In [12]:
query.process(raise_exception=True)

In [9]:
query.workplan

,path2file_aws,path2file_local,path2file_local_processed
2020-03-31 07:46:18,noaa-goes16/ABI-L2-ACMC/2020/091/07/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-31 07:51:18,noaa-goes16/ABI-L2-ACMC/2020/091/07/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-31 07:56:18,noaa-goes16/ABI-L2-ACMC/2020/091/07/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-31 08:01:18,noaa-goes16/ABI-L2-ACMC/2020/091/08/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-31 08:06:18,noaa-goes16/ABI-L2-ACMC/2020/091/08/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
...,...,...,...
2020-05-31 23:36:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-05-31 23:41:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-05-31 23:46:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-05-31 23:51:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...


In [9]:
query.workplan

,path2file_aws,path2file_local,path2file_local_processed
2020-03-20 03:31:14,noaa-goes16/ABI-L2-ACMC/2020/080/03/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-20 03:36:14,noaa-goes16/ABI-L2-ACMC/2020/080/03/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-20 03:41:14,noaa-goes16/ABI-L2-ACMC/2020/080/03/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-20 03:46:14,noaa-goes16/ABI-L2-ACMC/2020/080/03/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-20 03:51:14,noaa-goes16/ABI-L2-ACMC/2020/080/03/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
...,...,...,...
2020-05-31 23:36:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-05-31 23:41:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-05-31 23:46:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-05-31 23:51:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...


In [9]:
query.workplan

,path2file_aws,path2file_local,path2file_local_processed
2020-03-13 08:16:14,noaa-goes16/ABI-L2-ACMC/2020/073/08/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-13 08:21:14,noaa-goes16/ABI-L2-ACMC/2020/073/08/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-13 08:26:14,noaa-goes16/ABI-L2-ACMC/2020/073/08/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-13 08:31:14,noaa-goes16/ABI-L2-ACMC/2020/073/08/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-13 08:36:14,noaa-goes16/ABI-L2-ACMC/2020/073/08/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
...,...,...,...
2020-05-31 23:36:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-05-31 23:41:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-05-31 23:46:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-05-31 23:51:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...


In [11]:
query.workplan

,path2file_aws,path2file_local,path2file_local_processed
2020-03-07 10:21:14,noaa-goes16/ABI-L2-ACMC/2020/067/10/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-07 10:26:14,noaa-goes16/ABI-L2-ACMC/2020/067/10/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-07 10:31:14,noaa-goes16/ABI-L2-ACMC/2020/067/10/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-07 10:36:14,noaa-goes16/ABI-L2-ACMC/2020/067/10/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-07 10:41:14,noaa-goes16/ABI-L2-ACMC/2020/067/10/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
...,...,...,...
2020-05-31 23:36:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-05-31 23:41:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-05-31 23:46:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-05-31 23:51:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...


In [ ]:
24587

In [14]:
ds = xr.open_dataset('/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_CTP_projected2surfrad/ABI_L2_CTP_projected2surfrad_20200301_180114.nc')

In [15]:
ds

<xarray.Dataset>
Dimensions:                     (datetime: 1, radius: 5, stats: 3,
                                 data_quality: 1, site: 7)
Coordinates:
  * datetime                    (datetime) datetime64[ns] 2020-03-01T18:02:32...
  * radius                      (radius) int64 5 10 25 50 100
  * stats                       (stats) object 'median' 'mean' 'std'
  * site                        (site) object 'BND' 'SXF' 'TBL' ... 'GWN' 'PSU'
  * data_quality                (data_quality) object 'high'
Data variables:
    PRES                        (datetime, data_quality, radius, stats, site) float32 ...
    num_of_valid_points         (datetime, data_quality, radius, site) int64 ...
    PRES_on_pixel               (datetime, site) float32 ...
    DQF_on_pixel                (datetime, site) int64 ...
    argmin_x                    (datetime, site) float32 ...
    argmin_y                    (datetime, site) float32 ...
    lon_gritpoint               (datetime, site) float32 ...
    lat_gridpoint               (datetime, site) float32 ...
    lon_station                 (datetime, site) float32 ...
    lat_station                 (datetime, site) float32 ...
    distance_station_gridpoint  (datetime, site) float32 ...
Attributes:
    info:     This file contains a projection of satellite data onto SURFRAD ...

# Concatenate

In [6]:
reload(ngs)

<module 'nesdis_gml_synergy.satlab' from '/mnt/telg/prog/nesdis_gml_synergy/nesdis_gml_synergy/satlab.py'>

In [10]:
cc = ngs.Concatonator(path2scraped_files='/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_CTP_projected2surfrad/',
                 path2concat_files ='/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_CTP_projected2surfrad_concat/',
                 datetime_format='ABI_L2_CTP_projected2surfrad_%Y%m%d_%H%M%S.nc',
                 test = False,)

In [11]:
cc.workplan

,path2scraped_files,datetime,date,product_name,path2concat_files,p2rf_exists


In [9]:
ds=cc.concat_and_save(save=True, test=False, verbose=False)